# T-725 Natural Language Processing: Lab 2
In today's lab, we will be working with text classification.

To begin with, do the following:
* Select `"File" > "Save a copy in Drive"` to create a local copy of this notebook that you can edit.
* Select `"Runtime" > "Run all"` to run the code in this notebook.

## List comprehensions in Python
List comprehensions are a concise way of creating lists in Python, and take the form:

```python
[expression for item in iterable]
```

A list comprehension creates a new list by evaluating some expression for every item in a given iterable (such as a string, a list or a dictionary). Let's look at an example:

In [102]:
sentence = "In a hole in the ground there lived a hobbit."
words = sentence.split()
print(words)

# Example of a list comprehension
word_lengths = [len(word) for word in words]
print(word_lengths)

# This is equal to
word_lengths = []
for word in words:
  word_lengths.append(len(word))

print(word_lengths)

['In', 'a', 'hole', 'in', 'the', 'ground', 'there', 'lived', 'a', 'hobbit.']
[2, 1, 4, 2, 3, 6, 5, 5, 1, 7]
[2, 1, 4, 2, 3, 6, 5, 5, 1, 7]


You can also add a conditional statement to list comprehensions, so that the expression will only be evaluated for items that meet a certain criteria:

In [103]:
e_words = [word for word in words if len(word) > 5]
print(e_words)

['ground', 'hobbit.']


Python also has set and dictionary comprehensions:

In [104]:
lowercase_characters = {c.lower() for c in sentence}
print(lowercase_characters)

word_length = {word: len(word) for word in words}
print(word_length['ground'])

{'u', 'r', 'a', 'l', 'g', ' ', 'o', 't', 'v', 'h', '.', 'e', 'n', 'd', 'b', 'i'}
6


A nested list is a list within another list. You can iterate through nested lists in the following way:

In [105]:
# A list of countries and their capitals within different continents
continents = [
    [('Iceland', 'Reykjavík'), ('Germany', 'Berlin'), ('Spain', 'Madrid')],  # Europe
    [('Japan', 'Tokyo'), ('China', 'Beijing'), ('South Korea', 'Seoul')],  # Asia
    [('Nigeria', 'Abuja'), ('Algeria', 'Algiers'), ('Angola', 'Luanda')]  # Africa
]

# Create a list of all the countries in the previous list
[country for continent in continents for (country, capital) in continent]

['Iceland',
 'Germany',
 'Spain',
 'Japan',
 'China',
 'South Korea',
 'Nigeria',
 'Algeria',
 'Angola']

## Sentiment analysis with NLTK
[Chapter 6](https://www.nltk.org/book/ch06.html) of the NLTK book shows how the toolkit can be used to create document classifiers, including a sentiment analyzer. The NLTK includes the `movie_reviews` corpus, which contains 2,000 movie reviews. Half of the reviews have been labelled as **positive** and the other half as **negative**. Let's download it and take a look:

In [106]:
import nltk
from nltk.corpus import movie_reviews
nltk.download('punkt')

nltk.download('movie_reviews')
print("Categories:", movie_reviews.categories())

Categories: ['neg', 'pos']


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


As expected, there are two categories: `pos` for positive reviews and `neg` for negative reviews. For this particular corpus, each review is stored as a separate text file. To get a list of all the text files in the corpus, we can use `movie_reviews.fileids()`. We can also get a list of files for a specific category:

In [107]:
pos_fileids = movie_reviews.fileids('pos')
neg_fileids = movie_reviews.fileids('neg')

print(pos_fileids[:5])  # The first 5 positive reviews
print(neg_fileids[:5])  # The first 5 negative reviews

['pos/cv000_29590.txt', 'pos/cv001_18431.txt', 'pos/cv002_15918.txt', 'pos/cv003_11664.txt', 'pos/cv004_11636.txt']
['neg/cv000_29416.txt', 'neg/cv001_19502.txt', 'neg/cv002_17424.txt', 'neg/cv003_12683.txt', 'neg/cv004_12641.txt']


We can get a list of all the tokens in the corpus with `movie_reviews.words()`. We can also specify a filename to get a single tokenized review:

In [108]:
pos_reviews = [movie_reviews.words(fid) for fid in pos_fileids]
neg_reviews = [movie_reviews.words(fid) for fid in neg_fileids]


print(pos_reviews[0][:10])  # The first 10 tokens of the first positive review
print(neg_reviews[0][:10])  # The first 10 tokens of the first negative review

['films', 'adapted', 'from', 'comic', 'books', 'have', 'had', 'plenty', 'of', 'success']
['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party']


Some words, such as *brilliant* and *memorable*, are more strongly associated with positive reviews than negative ones. Similarly, *boring* and *unfunny* have a stronger association with negative reviews.

Using the movie review corpus, we can train a classifier to predict whether a given review is positive or negative. The classifier extracts a set of *features* from every review, which are then used to make the classification. In this case, the features we use will be a dictionary that tells us whether each of the 2,000 most common words in the corpus is present within a review or not.

In [109]:
# Create a set with 2,000 of the most frequent words in the movie review corpus
movie_fd = nltk.FreqDist(movie_reviews.words())
movie_words = {word for word, count in movie_fd.most_common(2000)}

# For a given review (in the form of a list or set of tokens), create a
# dictionary which tells us which words are present and which are not.
def get_review_features(review):
  review_words = set(review)
  return {word: word in review_words for word in movie_words}

In [110]:
# Let's see how this works for the first positive review:
example_features = get_review_features(pos_reviews[0])
print("'funny' is in the review:", example_features['funny'])
print("'boring' is in the review:", example_features['boring'])

'funny' is in the review: True
'boring' is in the review: False


Next, let's create a training set that we can use to train a Naive Bayesian classifier. The training set, in this case, is a list of tuples in the format `[(features, category), ...]`, where `features` is a dictionary from `get_review_features()` and `category` is either `pos` or `neg`, depending on whether the review is positive or negative. To get an idea of how well the classifier performs, we're going to reserve 10% of the reviews for testing. That means that we'll be training our classifier on 1800 examples and testing it on 200 examples.

In [187]:
pos_examples = [(get_review_features(review), 'pos') for review in pos_reviews]
neg_examples = [(get_review_features(review), 'neg') for review in neg_reviews]

movie_training = pos_examples[:900] + neg_examples[:900]  # 1800 examples total
movie_test = pos_examples[900:] + neg_examples[900:]  # 200 examples total

print(movie_training[0])

({'needs': True, 'edge': False, 'was': True, 'vegas': False, 'wrote': False, 'ice': False, 'control': False, 'rocky': False, 'steal': False, 'incredible': False, 'myers': False, 'himself': False, 'reason': False, 'violent': True, 'twists': False, 'used': False, 'minor': False, 'affleck': False, 'spawn': True, 'scientist': False, 'broken': False, 'him': True, 'novel': True, 'jason': False, 'material': False, 'match': False, 'begin': False, 'expectations': False, 'century': False, 'subject': True, 'west': False, 'mediocre': False, 'proves': False, 'can': True, 'directing': False, 'determined': False, 'thing': True, 'necessary': False, 'saving': False, 'stuart': False, 'became': False, 'exception': False, 'lee': False, 'else': False, 'struggle': False, 'suspense': False, 'master': False, 'latter': False, 'teenage': False, 'nights': False, 'directors': True, 'deal': False, 'so': True, 'steven': False, 'magic': False, 'nudity': False, 'these': False, 'media': False, 'x': False, 'mix': False

Now we have everything we need to train our classifier.

In [112]:
movie_classifier = nltk.NaiveBayesClassifier.train(movie_training)

How well does it perform on the test set?

In [113]:
print("Accuracy:", nltk.classify.accuracy(movie_classifier, movie_test))

Accuracy: 0.815


The classifier achieves an accuracy of 81.5%. Let's take a look at which words have the biggest weights:

In [114]:
movie_classifier.show_most_informative_features(20)

Most Informative Features
             outstanding = True              pos : neg    =     15.6 : 1.0
                   mulan = True              pos : neg    =      9.0 : 1.0
             wonderfully = True              pos : neg    =      7.1 : 1.0
                  seagal = True              neg : pos    =      7.0 : 1.0
                   damon = True              pos : neg    =      6.1 : 1.0
                   flynt = True              pos : neg    =      5.7 : 1.0
                  wasted = True              neg : pos    =      5.6 : 1.0
                    lame = True              neg : pos    =      5.3 : 1.0
                  poorly = True              neg : pos    =      5.2 : 1.0
                   awful = True              neg : pos    =      4.9 : 1.0
              ridiculous = True              neg : pos    =      4.8 : 1.0
                    jedi = True              pos : neg    =      4.4 : 1.0
                 unfunny = True              neg : pos    =      4.4 : 1.0

# Assignment
Answer the following questions and hand in your solution in Canvas before 23:59 on September 5th. Remember to save your file before uploading it.

## Question 1
The NLTK also includes a `subjectivity` corpus, which contains a collection of sentences that have either been categorized as **subjective** (emotional, expressing personal feelings and views)  or **objective** (more rational, factual). Some examples:

* **Objective sentences**:
  * uma thurman stars in quentin tarantino's fourth film venture , kill bill .  
  * he lives in a motor garage with his six friends .
  * the ensuing battle was one of the most savage in u . s . history .
* **Subjective sentences**:
  * seagal's strenuous attempt at a change in expression could very well clinch him this year's razzie .
  * de niro cries . you'll cry for your money back .
  * a heroic tale of persistence that is sure to win viewers' hearts .

Unlike the movie review corpus, where every review is stored in separate file, here there is only one file for each category.

Complete the following tasks:
1. Import and download the `subjectivity` corpus.
2. Find the names of each category.
3. Using the category names, get the relative path of each file.
4. Get a list of tokenized sentences for each category (using `subjectivity.sents(fileid)`).

In [115]:
from nltk.corpus import subjectivity

nltk.download("subjectivity")

print("Categories:", subjectivity.categories())

subj_fileids = subjectivity.fileids("subj")
obj_fileids = subjectivity.fileids("obj")

subj_tokens = subjectivity.sents(subj_fileids)
obj_tokens = subjectivity.sents(obj_fileids)

print(subj_tokens[:10])
print(obj_tokens[:10])

Categories: ['obj', 'subj']
[['smart', 'and', 'alert', ',', 'thirteen', 'conversations', 'about', 'one', 'thing', 'is', 'a', 'small', 'gem', '.'], ['color', ',', 'musical', 'bounce', 'and', 'warm', 'seas', 'lapping', 'on', 'island', 'shores', '.', 'and', 'just', 'enough', 'science', 'to', 'send', 'you', 'home', 'thinking', '.'], ['it', 'is', 'not', 'a', 'mass-market', 'entertainment', 'but', 'an', 'uncompromising', 'attempt', 'by', 'one', 'artist', 'to', 'think', 'about', 'another', '.'], ['a', 'light-hearted', 'french', 'film', 'about', 'the', 'spiritual', 'quest', 'of', 'a', 'fashion', 'model', 'seeking', 'peace', 'of', 'mind', 'while', 'in', 'a', 'love', 'affair', 'with', 'a', 'veterinarian', 'who', 'is', 'a', 'non-practicing', 'jew', '.'], ['my', 'wife', 'is', 'an', 'actress', 'has', 'its', 'moments', 'in', 'looking', 'at', 'the', 'comic', 'effects', 'of', 'jealousy', '.', 'in', 'the', 'end', ',', 'though', ',', 'it', 'is', 'only', 'mildly', 'amusing', 'when', 'it', 'could', 'have'

[nltk_data] Downloading package subjectivity to /root/nltk_data...
[nltk_data]   Package subjectivity is already up-to-date!


## Question 2
Complete the following tasks:
1. Create a set with the 2,000 most common words in the `subjectivity` corpus using `nltk.FreqDist()`.
2. Create a function that takes a single, tokenized sentence as input (e.g., `['the', 'ensuing', 'battle', ...]`), and returns a dictionary of the 2,000 most frequent words and whether or not they are in the sentence (e.g., `{'battle': True, 'amusing': False, ...}`).

In [116]:
subjectivity_fd = nltk.FreqDist(subjectivity.words())
subjectivity_wc = {word for word, count in subjectivity_fd.most_common(2000)}

def get_presence_in_senteces(sentence):
  sentence_words = set(sentence)
  return {word: word in sentence_words for word in subjectivity_wc}

example_features = get_presence_in_senteces(subj_tokens[0])

print(subj_tokens[0])

key = 'about'

print("'" + key + "' is in the review:", example_features[key])



['smart', 'and', 'alert', ',', 'thirteen', 'conversations', 'about', 'one', 'thing', 'is', 'a', 'small', 'gem', '.']
'about' is in the review: True


## Question 3
Complete the following tasks:
1. Create a training set with 9,000 sentences (4,500 of each category)
2. Create a test set with 1,000 sentences (500 of each category)

In [117]:
subj_examples = [(get_presence_in_senteces(sentence),'subj') for sentence in subj_tokens]
obj_examples = [(get_presence_in_senteces(sentence),'obj') for sentence in obj_tokens]

sentence_training = subj_examples[:4500] + obj_examples[:4500]
sentence_test = subj_examples[4500:] + obj_examples[4500:]

## Question 4
Complete the following tasks:
1. Train a Naive Bayes classifier using the training set from the previous question.
2. Evaluate the classifier on the test set. How accurate is it?
3. Find the 20 most informative features.

In [118]:
subjectivity_classifier = nltk.NaiveBayesClassifier.train(sentence_training)
print("Accuracy:", nltk.classify.accuracy(subjectivity_classifier, sentence_test))

subjectivity_classifier.show_most_informative_features(20)

Accuracy: 0.906
Most Informative Features
                      -- = True             subj : obj    =     70.1 : 1.0
                   order = True              obj : subj   =     39.0 : 1.0
                 decides = True              obj : subj   =     35.7 : 1.0
                  sister = True              obj : subj   =     27.7 : 1.0
            entertaining = True             subj : obj    =     26.6 : 1.0
              girlfriend = True              obj : subj   =     26.3 : 1.0
                discover = True              obj : subj   =     25.0 : 1.0
                  film's = True             subj : obj    =     25.0 : 1.0
                  you're = True             subj : obj    =     22.6 : 1.0
                daughter = True              obj : subj   =     22.4 : 1.0
                 married = True              obj : subj   =     21.7 : 1.0
                 amusing = True             subj : obj    =     19.7 : 1.0
                   plans = True              obj : subj   

# Question 5
Dialog acts are sort of the type of *action* performed by the speaker. In the instant messaging corpus dataset 'NPS', each utterance is labeled with one of 15 dialogue act types, such as **Statement**, **Emotion**, **ynQuestion**, **Continuer**, etc.

Your task is to classify text from the NPS corpus into two dialog acts: **whQuestion** or **Emotion**.

Start by downloading the NPS corpus and getting all posts from the corpus:

In [119]:
nltk.download('nps_chat')
posts = nltk.corpus.nps_chat.xml_posts()

[nltk_data] Downloading package nps_chat to /root/nltk_data...
[nltk_data]   Package nps_chat is already up-to-date!


Create a list that only includes posts of class **Emotion** and **whQuestion**. You can access the class of a post by calling `post.get("class")`.

In [176]:
post_emotion = [post for post in posts if post.get('class') == "Emotion" ]
post_question = [post for post in posts if post.get('class') == "whQuestion" ]

posts_list = post_emotion + post_question

1106
533


Randomize the posts and create a training set and a test set, where the first 1300 **Emotion + whQuestion** posts are used for training and the rest for testing.

In [182]:
import random

random.shuffle(posts_list)

posts_training = posts_list[:1300]
posts_test = posts_list[1300:]

print(posts_list[:5])

[<Element 'Post' at 0x7f05e03b7ec0>, <Element 'Post' at 0x7f05ddfea9d0>, <Element 'Post' at 0x7f05ddf3d1c0>, <Element 'Post' at 0x7f05ddfeb060>, <Element 'Post' at 0x7f05dd795d00>]


Create a list of the 200 most frequent tokens in the training set. You can access the text of a `post` object by calling `post.text`. Remember that the **split** function will use whitespace to tokenize a string: `some_string.split()`

In [191]:
post_texts = [post.text.split() for post in posts_training]
post_words = [item for sublist in post_texts for item in sublist]

post_fd = nltk.FreqDist(post_words)
post_wc = {word for word, count in post_fd.most_common(200)}

print(post_wc)

{'im', 'who', '11-09-40sUser7', 'from?', '10-24-40sUser13', '10-19-30sUser31', 'happened', 'omg', ':tongue:', 'should', 'was', '11-08-20sUser21', '11-09-40sUser49', 'where?', 'lol,', 'laffs', 'what?', '10-19-20sUser121', 'everyone', 'were', 'much', 'me', 'come', '10-19-30sUser28', 'are', 'up', 'we', "who's", 'girl', 'doing', 'I', 'where', 'hehe', 'him', '10-19-30sUser17', 'ok', 'oh', ':-o', '10-19-20sUser7', 'back', 'wtf', '@', 'Lmfao', ')))))))))))))', '10-19-adultsUser1', 'time', 'this', 'go', '11-09-20sUser156', 'wants', 'o0', 'am', 'the', '10-19-adultsUser28?', ')))))))))))', ':-@', 'about', 'been', 'Lol', 'all', 'if', 'did', 'well', 'no', 'ya', 'what', 'of', '10-19-adultsUser32', 'here', '.', '10-19-40sUser3', '10-24-40sUser19', ';)', '10-19-adultsUser28', 'do', 'you?', 'sang', 'when', 'whos', ':)', 'so', 'how', ':-)', 'Who', '10-19-30sUser11', '??', 'like', 'kinda', 'my', '10-24-40sUser6', 'goin', "what's", 'Hi', 'and', 'get', 'o', 'always', 'day', '11-09-adultsUser72', '10-19-ad


Define two feature selection functions that take a string as input and output a dictionary of features:
* `get_word_features(string)`
* `get_custom_features(string)`

Begin by defining `get_word_features`. This function should use the words as features, just like in the movie review example above.




In [183]:
def get_word_features(word):
  post_set = set(word)
  return {word: word in post_set for word in post_wc}

example_features = get_word_features(post_texts[0])

print(post_texts[0])

key = 'how'

print("'" + key + "' is in the message:", example_features[key])

[':)']
'how' is in the message: False


Next, define `get_custom_features`. This function should extract the features from the text that characterize the **Emotion** and **whQuestions** classes.

In [197]:
def get_custom_features(word):
  custom_features = {'lmao', ':)','what','?','!!!!!!!','lol','haha','when',
                     'why','how','who','where','omg', 'wtf', 'hehe', ':-o',
                     'ok', 'I', '@'}
  post_set = set(word)
  return {word: word in post_set for word in custom_features}

example_features = get_custom_features(post_texts[0])

print(post_texts[0])

key = 'lol'

print("'" + key + "' is in the message:", example_features[key])


['lol']
'lol' is in the message: True


Conduct the following tasks:
*   Train two Naive Bayes classifiers on the **Emotion + whQuestions** training set: one that uses the `get_word_features` function and another using `get_custom_features`.
*   Evaluate each classifier on the test set. How accurate are they? Which one is better?
*   What are the 20 most informative features for each classifier?


In [196]:
emotion_training_examples = [(get_word_features(sentence.text.split()), sentence.get("class")) for sentence in posts_training if sentence.get("class") == "Emotion"]
wh_training_examples = [(get_word_features(sentence.text.split()), sentence.get("class")) for sentence in posts_training if sentence.get("class") == "whQuestion"]

feature_examples_training = emotion_training_examples + wh_training_examples
random.shuffle(feature_examples_training)

emotion_testing_examples = [(get_word_features(sentence.text.split()), sentence.get("class")) for sentence in posts_test if sentence.get("class") == "Emotion"]
wh_testing_examples = [(get_word_features(sentence.text.split()), sentence.get("class")) for sentence in posts_test if sentence.get("class") == "whQuestion"]

feature_examples_testing = emotion_testing_examples + wh_testing_examples
random.shuffle(feature_examples_testing)

first_post_classifier = nltk.NaiveBayesClassifier.train(feature_examples_training)
print("Accuracy (feature word model):", nltk.classify.accuracy(first_post_classifier, feature_examples_testing))

first_post_classifier.show_most_informative_features(20)


emotion_training_examples = [(get_custom_features(sentence.text.split()), sentence.get("class")) for sentence in posts_training if sentence.get("class") == "Emotion"]
wh_training_examples = [(get_custom_features(sentence.text.split()), sentence.get("class")) for sentence in posts_training if sentence.get("class") == "whQuestion"]

feature_examples_training = emotion_training_examples + wh_training_examples
random.shuffle(feature_examples_training)

emotion_testing_examples = [(get_custom_features(sentence.text.split()), sentence.get("class")) for sentence in posts_test if sentence.get("class") == "Emotion"]
wh_testing_examples = [(get_custom_features(sentence.text.split()), sentence.get("class")) for sentence in posts_test if sentence.get("class") == "whQuestion"]

feature_examples_testing = emotion_testing_examples + wh_testing_examples
random.shuffle(feature_examples_testing)

first_post_classifier = nltk.NaiveBayesClassifier.train(feature_examples_training)
print("Accuracy (feature custom model):", nltk.classify.accuracy(first_post_classifier, feature_examples_testing))

first_post_classifier.show_most_informative_features(20)



Accuracy (feature word model): 0.9351032448377581
Most Informative Features
                    what = True           whQues : Emotio =     98.0 : 1.0
                     how = True           whQues : Emotio =     82.5 : 1.0
                      is = True           whQues : Emotio =     61.3 : 1.0
                       u = True           whQues : Emotio =     51.5 : 1.0
                     you = True           whQues : Emotio =     33.1 : 1.0
                      in = True           whQues : Emotio =     30.3 : 1.0
                     lol = True           Emotio : whQues =     28.8 : 1.0
                     the = True           whQues : Emotio =     28.7 : 1.0
                     and = True           whQues : Emotio =     27.5 : 1.0
                    that = True           whQues : Emotio =     21.9 : 1.0
                      to = True           whQues : Emotio =     18.4 : 1.0
                     LOL = True           Emotio : whQues =     11.2 : 1.0
                    lmao